In [23]:
import pandas as pd
import numpy as np

In [24]:
train = pd.read_csv('../data/train_store_combined.csv')
test = pd.read_csv('../data/test_store_combined.csv')

In [25]:
def isWeekend(x):
    if x<6:
        return 0
    else: 
        return 1
train["weekend"]= train["DayOfWeek"].apply(isWeekend )

In [26]:
def startMidEndMonth(x):
    if x<10:
        return 0
    elif x<20:
        return 1
    else:
        return 2

In [27]:
train["MonthState"]=train["Day"].apply(startMidEndMonth)

In [28]:
import bisect
train["Date"]=pd.to_datetime(train["Date"])
dates=np.array(train[train["StateHoliday"]!="0"]["Date"].unique())
dates=np.sort(dates)
a=train["Date"].iloc[119]

index = bisect.bisect(dates, a)
print(index,dates[index-1],dates[index],a)

32 2015-04-03T00:00:00.000000000 2015-04-06T00:00:00.000000000 2015-04-03 00:00:00


In [29]:
def datToAndAfterHoliday(df,Column,holidays):
    to=[]
    after=[]
    for a in df[Column]:
        index=bisect.bisect(holidays,a)
        if len(holidays)==index:
            to.append(pd.Timedelta(0, unit='d') )
            after.append(a - holidays[index-1])
        else:
            to.append(holidays[index] - a)
            after.append(a -holidays[index-1])
    return to,after

In [30]:
train["To"],train["After"]=datToAndAfterHoliday(train,"Date",dates)

In [31]:
train['Promo2SinceWeek'] = train['Promo2SinceWeek'].fillna(0)
train['Promo2SinceYear'] = train['Promo2SinceYear'].fillna(0)
train['PromoInterval'] = train['PromoInterval'].fillna(train['PromoInterval'].mode().iloc[0])

train['CompetitionDistance'] = train['CompetitionDistance'].fillna(train['CompetitionDistance'].max())
train['CompetitionOpenSinceMonth'] = train['CompetitionOpenSinceMonth'].fillna(train['CompetitionOpenSinceMonth'].mode().iloc[0])
train['CompetitionOpenSinceYear'] = train['CompetitionOpenSinceYear'].fillna(train['CompetitionOpenSinceYear'].mode().iloc[0])